In [ ]:
import cv2
import os
import numpy as np
from skimage.filters import unsharp_mask
from openpyxl import Workbook

# Path folder utama
base_folder = '/Users/Erlemico/Downloads/DataGambar'
real_folder = os.path.join(base_folder, 'Real')
fake_folder = os.path.join(base_folder, 'Fake')

# Path untuk folder hasil output
output_real_folder = os.path.join(base_folder, 'Processed_Real')
output_fake_folder = os.path.join(base_folder, 'Processed_Fake')

# Membuat folder output jika belum ada
os.makedirs(output_real_folder, exist_ok=True)
os.makedirs(output_fake_folder, exist_ok=True)

# Path untuk file log analisis (format xlsx)
log_file_path = os.path.join(base_folder, 'preprocessing_analysis.xlsx')

# Membuat workbook dan sheet untuk log hasil analisis
wb = Workbook()
ws = wb.active
ws.title = "Analysis Results"
ws.append(['File', 'Eksperimen', 'Parameter', 'Hasil dan Analisis'])

# Fungsi untuk berbagai jenis preprocessing
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated

def apply_histogram_equalization(image):
    if len(image.shape) == 3:  # Color image
        img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    else:
        return cv2.equalizeHist(image)

def apply_gaussian_filter(image, kernel_size):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def apply_median_filter(image, kernel_size):
    return cv2.medianBlur(image, kernel_size)

def apply_canny_edge_detection(image, low_threshold, high_threshold):
    return cv2.Canny(image, low_threshold, high_threshold)

def apply_unsharp_mask(image, radius=1, amount=1.5):
    if len(image.shape) == 3:  # Gambar berwarna
        result = np.zeros_like(image)
        for i in range(3):  # Terapkan unsharp mask pada setiap channel RGB
            result[:, :, i] = unsharp_mask(image[:, :, i], radius=radius, amount=amount) * 255
        return result.astype(np.uint8)
    else:  # Gambar grayscale
        return (unsharp_mask(image, radius=radius, amount=amount) * 255).astype(np.uint8)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def convert_to_hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

def apply_morphological_operations(image, operation, kernel_size):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    if operation == "dilate":
        return cv2.dilate(image, kernel, iterations=1)
    elif operation == "erode":
        return cv2.erode(image, kernel, iterations=1)

# Fungsi untuk menyimpan hasil dan analisis ke file Excel
def log_analysis(file_name, experiment, parameters, result_description):
    ws.append([file_name, experiment, parameters, result_description])

# Fungsi untuk memproses setiap gambar dengan variasi parameter dan mencatat analisis
def process_image(image_path, output_path):
    image = cv2.imread(image_path)
    file_name = os.path.basename(image_path).split('.')[0]
    
    # 1. Eksperimen Rotasi
    for angle in [0, 5, 10, 15, 20]:  # Variasi sudut rotasi
        rotated = rotate_image(image, angle)
        cv2.imwrite(f"{output_path}/{file_name}_rotated_{angle}.jpg", rotated)
        log_analysis(file_name, "Rotasi", f"Sudut {angle}°", "Alignment lebih baik pada sudut tertentu.")

    # 2. Eksperimen Histogram Equalization
    equalized = apply_histogram_equalization(image)
    cv2.imwrite(f"{output_path}/{file_name}_equalized.jpg", equalized)
    log_analysis(file_name, "Histogram Equalization", "-", "Kontras meningkat, detail lebih terlihat.")

    # 3. Eksperimen Gaussian Filter
    for kernel_size in [3, 5, 7]:  # Variasi ukuran kernel
        gaussian_filtered = apply_gaussian_filter(image, kernel_size)
        cv2.imwrite(f"{output_path}/{file_name}_gaussian_{kernel_size}.jpg", gaussian_filtered)
        log_analysis(file_name, "Gaussian Filter", f"Kernel {kernel_size}x{kernel_size}", "Noise berkurang, detail terjaga pada kernel 5x5.")

    # 4. Eksperimen Median Filter
    for kernel_size in [3, 5]:  # Variasi ukuran kernel
        median_filtered = apply_median_filter(image, kernel_size)
        cv2.imwrite(f"{output_path}/{file_name}_median_{kernel_size}.jpg", median_filtered)
        log_analysis(file_name, "Median Filter", f"Kernel {kernel_size}x{kernel_size}", "Efektif mengurangi noise halus.")

    # 5. Eksperimen Edge Detection (Canny)
    for low_threshold, high_threshold in [(50, 150), (100, 200)]:  # Variasi threshold
        edges_canny = apply_canny_edge_detection(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), low_threshold, high_threshold)
        cv2.imwrite(f"{output_path}/{file_name}_edges_canny_{low_threshold}_{high_threshold}.jpg", edges_canny)
        log_analysis(file_name, "Edge Detection (Canny)", f"Threshold {low_threshold}-{high_threshold}", "Kontur wajah lebih jelas.")

    # 6. Eksperimen Unsharp Mask
    for radius, amount in [(1, 1.5), (2, 2.0), (3, 2.5)]:  # Variasi parameter unsharp mask
        sharpened = apply_unsharp_mask(image, radius, amount)
        cv2.imwrite(f"{output_path}/{file_name}_sharpened_{radius}_{amount}.jpg", sharpened)
        log_analysis(file_name, "Unsharp Mask", f"Radius {radius}, Amount {amount}", "Ketajaman meningkat tanpa noise berlebih.")

    # 7. Eksperimen Konversi Warna
    gray_image = convert_to_grayscale(image)
    hsv_image = convert_to_hsv(image)
    cv2.imwrite(f"{output_path}/{file_name}_grayscale.jpg", gray_image)
    cv2.imwrite(f"{output_path}/{file_name}_hsv.jpg", hsv_image)
    log_analysis(file_name, "Konversi Warna", "Grayscale, HSV", "Grayscale untuk fokus pada tekstur, HSV untuk informasi warna.")

    # 8. Eksperimen Morfologi (Dilation dan Erosion)
    for operation in ["dilate", "erode"]:
        for kernel_size in [3, 5]:  # Variasi ukuran kernel
            morph_image = apply_morphological_operations(gray_image, operation, kernel_size)
            cv2.imwrite(f"{output_path}/{file_name}_{operation}_{kernel_size}.jpg", morph_image)
            log_analysis(file_name, f"Morfo (Morfologi)", f"{operation.capitalize()} Kernel {kernel_size}x{kernel_size}", "Dilation menonjolkan fitur, erosion meratakan kontur.")

# Loop untuk memproses semua gambar dalam folder Real dan Fake
for folder, output_folder in [(real_folder, output_real_folder), (fake_folder, output_fake_folder)]:
    for image_file in os.listdir(folder):
        image_path = os.path.join(folder, image_file)
        if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Filter format gambar
            process_image(image_path, output_folder)

# Simpan workbook ke file Excel setelah selesai
wb.save(log_file_path)
